In [ ]:
from helper import *

import pandas as pd
import numpy as np
import os
import csv
import sys
import matplotlib
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [16, 8]
# use LaTeX fonts in the plot
matplotlib.use("pgf")
matplotlib.rcParams.update({
    "pgf.texsystem": "pdflatex",
    'font.family': 'serif',
    'text.usetex': True,
    'pgf.rcfonts': False,
})
%matplotlib inline

In [ ]:
# simply put osm files into the data-dir (all will get evaluated)
EVAL_DIR = "/home/felix/todo/osm-tmp"
MLP_METHODS = ["kmeans", "merge", "hop"]
# MLP_LEVELS = [[int(2 ** i)] for i in np.arange(8.0, 13.5, 0.5)]
MLP_LEVELS = [[int(2 ** i)] for i in np.arange(8.0, 13.5, 1.0)]
FAST_QUERY_METHODS = ["crp", "pch", "prp"]
QUERY_METHODS = ["normal", "bi"] + FAST_QUERY_METHODS
AREAS = ["saarland"]
print(MLP_METHODS, "with", MLP_LEVELS)

# Time

In [ ]:
df = pd.DataFrame()
for area in AREAS:
    for query in QUERY_METHODS:
        for mlp in MLP_METHODS:
            for partitions in MLP_LEVELS:
                if query == "normal" or query == "bi":
                    tmp = pd.read_json("/home/felix/todo/osm-tmp/" + area + "-" + query + "-time.json")
                else:
                    tmp = pd.read_json("/home/felix/todo/osm-tmp/" + area + "-" + mlp + "-" + "_".join(map(str, partitions)) + "-" + query + "-time.json")
                tmp["Area"] = area
                tmp["Query"] = query
                tmp["Mlp"] = mlp
                tmp["Mlp_partitions"] = "_".join(map(str, partitions))
                df = df.append(tmp, ignore_index = True)

In [ ]:
data = list()
labels = list()
for area in AREAS:
    for query in FAST_QUERY_METHODS:
        for mlp in MLP_METHODS:
            for partitions in MLP_LEVELS:
                tmp = df[(df.Area == area) & (df.Query == query) & (df.Mlp == mlp) & (df.Mlp_partitions == "_".join(map(str, partitions)))]
                data.append(tmp["time"])
                labels.append(query + "-" + area + "-" + "_".join(map(str, partitions)) + "-" + str(mlp))
        
fig, ax = plt.subplots()
ax.boxplot(data, labels=labels)#, showmeans=True)
plt.xticks(rotation=40, ha='right')
fig.savefig("comparison.pgf", bbox_inches="tight")

In [ ]:
fig, ax = plt.subplots()
speedups = list()
for area in AREAS:
    dijkstra = df[(df.Query == "normal") & (df.Area == area)]["time"].mean()
    bidijkstra = df[(df.Query == "bi") & (df.Area == area)]["time"].mean()
    for query in FAST_QUERY_METHODS:
        for mlp in MLP_METHODS:
            x = list()
            y = list()
            for partitions in MLP_LEVELS:
                tmp = df[(df.Area == area) & (df.Query == query) & (df.Mlp == mlp) & (df.Mlp_partitions == "_".join(map(str, partitions)))]
                x.append(partitions[0])
                y.append(tmp["time"].mean())
                speedups.append({"Query": query, "MLP": mlp, "_".join(map(str, partitions)): dijkstra / tmp["time"].mean()})
            plt.plot(x, y, marker=plot_get(query), color=plot_get(mlp), label=query + "-" + mlp, alpha=0.7)
plt.xlabel("MLP-Partition-Size")
plt.ylabel("Query-time")
plt.legend(loc='upper left')
fig.savefig("single-level.pgf", bbox_inches="tight")

In [ ]:
df_speedups = pd.DataFrame(speedups)
df_speedups = df_speedups.groupby(["Query", "MLP"]).first()
# df_speedups.reset_index(inplace=True)
# df_speedups.to_latex('speedups.tex', float_format="{:0.1f}".format, index=False)
latex = df_speedups.to_latex(float_format="{:0.1f}".format)
df_speedups

In [ ]:
# fixup ugly latex code to have single line header
latex_list = latex.splitlines()

latex_list[0] = latex_list[0].replace('ll', 'll|', 1)

partitions = latex_list[2]
headers = latex_list[3].split("&")
headers = [headers[0], headers[1], " \multicolumn{" + str(len(headers) - 2) + "}{c}{amount of partitions} \\\\"]

latex_list[3] = partitions
latex_list[2] = "&".join(headers)

latex_list.insert(len(latex_list)-8, '\midrule')
latex_list.insert(len(latex_list)-5, '\midrule')
latex_new = '\n'.join(latex_list)
with open("speedups.tex", "w") as latex_file:
    latex_file.writelines(latex_new)

# Count

In [ ]:

df = pd.DataFrame()
for area in AREAS:
    for query in QUERY_METHODS:
        for mlp in MLP_METHODS:
            for partitions in MLP_LEVELS:
                if query == "normal" or query == "bi":
                    tmp = pd.read_json("/home/felix/todo/osm-tmp/" + area + "-" + query + "-count.json")
                else:
                    tmp = pd.read_json("/home/felix/todo/osm-tmp/" + area + "-" + mlp + "-" + "_".join(map(str, partitions)) + "-" + query + "-count.json")
                tmp["Query"] = query
                tmp["Area"] = area
                tmp["Mlp"] = mlp
                tmp["Mlp_partitions"] = "_".join(map(str, partitions))
                df = df.append(tmp, ignore_index = True)

In [ ]:
fig, ax = plt.subplots()
for area in AREAS:
    for query in FAST_QUERY_METHODS:
        for mlp in MLP_METHODS:
            x = list()
            y = list()
            for partitions in MLP_LEVELS:
                tmp = df[(df.Area == area) & (df.Query == query) & (df.Mlp == mlp) & (df.Mlp_partitions == "_".join(map(str, partitions)))]
                x.append(partitions[0])
                y.append(tmp["heap_pops"].mean())
            plt.plot(x, y, marker=plot_get(query), color=plot_get(mlp), label=query + "-" + mlp, alpha=0.7)

plt.xlabel("MLP-Partition-Size")
plt.ylabel("heap-pops")
plt.legend(loc='upper left')
fig.savefig("query-heap_pops.pgf", bbox_inches="tight")

In [ ]:
fig, ax = plt.subplots()
for area in AREAS:
    for query in FAST_QUERY_METHODS:
        for mlp in MLP_METHODS:
            x = list()
            y = list()
            for partitions in MLP_LEVELS:
                tmp = df[(df.Area == area) & (df.Query == query) & (df.Mlp == mlp) & (df.Mlp_partitions == "_".join(map(str, partitions)))]
                x.append(partitions[0])
                y.append(tmp["relaxed_edges"].mean())
            plt.plot(x, y, marker=plot_get(query), color=plot_get(mlp), label=query + "-" + mlp, alpha=0.7)

plt.xlabel("MLP-Partition-Size")
plt.ylabel("relaxed-edges")
plt.legend(loc='upper left')
fig.savefig("query-relaxed_edges.pgf", bbox_inches="tight")